<a href="https://colab.research.google.com/github/znumunz/Final_DataViz/blob/main/Test_Final_DataViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# นำเข้าข้อมูล

ให้สร้างช่องทางลัดของไดร์ฟนี้ เข้าไปใน MyDrive ของตัวเอง ถึงจะสามารถ load ข้อมูลแบบไม่ต้องเปลี่ยน path ได้

https://drive.google.com/drive/folders/1ZkTs3dlvqGnfMPss3mS5xoCnB1NXep-y?usp=sharing

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Data_Final_DataViz'

In [ ]:
import pandas as pd

In [ ]:
# ทดสอบ
data1 = "stores (1)"
data2 =

df1 = pd.read_csv(base_path +"/"+ data1+".csv")
df2 =

df1.head()

# ส่งออก google sheets

In [ ]:
# ============================================================
# 🚀 Upload large DataFrames to separate Google Sheets (downsize + batch)
# ============================================================

!pip install gspread gspread_dataframe --quiet
from google.colab import auth
auth.authenticate_user()

import gspread
import pandas as pd
from google.auth import default
from gspread_dataframe import set_with_dataframe
import math

# ============================================================
# ✅ เชื่อมต่อ Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# ============================================================
# ✅ กำหนดชื่อไฟล์และ DataFrame ที่จะอัปโหลด
sheets_info = {
    "Fianl_DataViz_Data1": Fianl_DataViz_Data,
    "Fianl_DataViz_Data3": Fianl_DataViz_Data,
    }

# ============================================================
# ✅ ตั้งค่า batch size (จำนวนแถวต่อครั้ง)
BATCH_SIZE = 50000

# ============================================================
# ฟังก์ชันลดขนาด DataFrame
def downsize_df(df):
    # ลด int64 → int32
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    # ลด float64 → float32
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    # แปลง datetime → string
    for col in df.select_dtypes(include=['datetime', 'datetimetz']).columns:
        df[col] = df[col].astype(str)
    return df

# ============================================================
# อัปโหลดทีละ DataFrame
for sheet_title, df in sheets_info.items():
    print(f"\n📤 กำลังอัปโหลด '{sheet_title}'...")

    df = downsize_df(df)  # ลดขนาดข้อมูล
    total_rows = df.shape[0]
    num_batches = math.ceil(total_rows / BATCH_SIZE)

    # สร้าง Google Sheet ใหม่
    sh = gc.create(sheet_title)
    sh.share(None, perm_type='anyone', role='writer')  # ใครก็เข้าดูได้
    ws = sh.sheet1

    print(f"ℹ️ Total rows: {total_rows}, will upload in {num_batches} batches of {BATCH_SIZE}")

    # อัปโหลด batch แรก พร้อม header
    batch = df.iloc[0:min(BATCH_SIZE, total_rows)]
    set_with_dataframe(ws, batch, include_index=False, include_column_header=True)

    # อัปโหลด batch ถัดไป (ถ้ามี)
    for i in range(1, num_batches):
        start_row = i * BATCH_SIZE
        end_row = min((i+1)*BATCH_SIZE, total_rows)
        batch = df.iloc[start_row:end_row]
        set_with_dataframe(ws, batch, include_index=False, include_column_header=False, row=start_row+1)
        print(f"⏳ Uploaded rows {start_row} to {end_row}")

    print(f"✅ Upload finished: '{sheet_title}' ({df.shape[0]} rows, {df.shape[1]} cols)")
    print(f"🔗 Google Sheets URL: https://docs.google.com/spreadsheets/d/{sh.id}")

print("\n🎉 อัปโหลดครบทุกชุดข้อมูลเรียบร้อยแล้ว!")
